In [27]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn import set_config
from catboost import CatBoostRegressor
from datetime import timedelta
from lightgbm import LGBMRegressor
import lightgbm as lgb
import optuna
from optuna.distributions import CategoricalDistribution, IntDistribution, FloatDistribution
from optuna.integration import OptunaSearchCV
from optuna.integration.lightgbm import LightGBMTunerCV
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from category_encoders import TargetEncoder, BinaryEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from category_encoders import TargetEncoder, BinaryEncoder, WOEEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from category_encoders.cat_boost import CatBoostEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import log_loss
from catboost import CatBoostClassifier, Pool, cv
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.model_selection import StratifiedKFold

In [19]:
train = pd.read_csv('train.csv', encoding='cp949').drop(columns='index').iloc[:,0:18]
target = pd.read_csv('train.csv', encoding='cp949').drop(columns='index').iloc[:,-1]
test = pd.read_csv('test.csv', encoding='cp949').drop(columns='index').iloc[:,0:18]

In [3]:
df = pd.concat([train, test])

In [4]:
target = target.astype('object')

In [5]:
df = df.fillna({'occyp_type':'No job'})

In [7]:
df['Employed']= df['DAYS_EMPLOYED'].map(lambda x: 0 if x>0 else x)
df['Employed'] = abs(df['Employed'])/360
df = df.astype({'Employed': 'int'})

In [8]:
df['Age'] = abs(df['DAYS_BIRTH'])/360
df = df.astype({'Age': 'int'})

In [9]:
df['Month'] = abs(df['begin_month'])
df = df.astype({'Month': 'int'})

In [10]:
df

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,...,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,Age,Employed,Month
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,...,1,0,0,0,No job,2.0,-6.0,38,13,6
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,...,1,0,0,1,Laborers,3.0,-5.0,31,4,5
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,...,1,0,1,0,Managers,2.0,-22.0,53,12,22
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,...,1,0,1,0,Sales staff,2.0,-37.0,41,5,37
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,...,1,0,0,0,Managers,2.0,-26.0,41,5,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,F,Y,Y,0,202500.0,Working,Incomplete higher,Married,House / apartment,-18593,...,1,1,1,0,Accountants,2.0,-19.0,51,15,19
9996,M,Y,Y,0,202500.0,Working,Secondary / secondary special,Civil marriage,House / apartment,-10886,...,1,1,0,0,Laborers,2.0,-34.0,30,3,34
9997,F,N,Y,0,292500.0,Working,Secondary / secondary special,Married,House / apartment,-21016,...,1,0,0,0,Medicine staff,2.0,-55.0,58,38,55
9998,F,Y,N,0,180000.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-16541,...,1,0,1,0,No job,2.0,-33.0,45,3,33


In [11]:
df = df.drop(columns=['DAYS_EMPLOYED','DAYS_BIRTH','begin_month','FLAG_MOBIL'])

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36457 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   gender        36457 non-null  object 
 1   car           36457 non-null  object 
 2   reality       36457 non-null  object 
 3   child_num     36457 non-null  int64  
 4   income_total  36457 non-null  float64
 5   income_type   36457 non-null  object 
 6   edu_type      36457 non-null  object 
 7   family_type   36457 non-null  object 
 8   house_type    36457 non-null  object 
 9   work_phone    36457 non-null  int64  
 10  phone         36457 non-null  int64  
 11  email         36457 non-null  int64  
 12  occyp_type    36457 non-null  object 
 13  family_size   36457 non-null  float64
 14  Age           36457 non-null  int32  
 15  Employed      36457 non-null  int32  
 16  Month         36457 non-null  int32  
dtypes: float64(2), int32(3), int64(4), object(8)
memory usage: 4.6+ MB


In [13]:
train = df[:26457]
test = df[26457:]

In [14]:
numeric_features = train.dtypes[train.dtypes != "object"].index.tolist()
categorical_features = train.dtypes[train.dtypes == "object"].index.tolist()

train = train[numeric_features+categorical_features]
test = test[numeric_features+categorical_features]

['gender',
 'car',
 'reality',
 'income_type',
 'edu_type',
 'family_type',
 'house_type',
 'occyp_type']

In [15]:
def remove_outlier(X, q=0.05):  
    df1 = pd.DataFrame(X)
    return df1.apply(lambda x: x.clip(x.quantile(q), x.quantile(1-q)), axis=0).values

numeric_transformer = Pipeline(
    steps=[
        #("imputer", SimpleImputer(strategy="median")),
        ("outlier", FunctionTransformer(remove_outlier, kw_args={'q':0.05})),
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        #("imputer", SimpleImputer(strategy="most_frequent")), 
        ("encoder", OneHotEncoder(handle_unknown="ignore", sparse=True)),
        #("encoder", TargetEncoder(smoothing=10, min_samples_leaf=15)),
        #("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ]
)

column_transformer = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

preprocessor = Pipeline(
    steps=[
        ("column", column_transformer), 
        ("selector", SelectPercentile(percentile=100)),
    ]
)

In [25]:
n_est = 2000
seed = 42
n_fold = 15
n_class = 3

X = train
y = target
X_test = test

In [28]:
skfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
folds=[]
for train_idx, valid_idx in skfold.split(X, y):
        folds.append((train_idx, valid_idx))

cat_pred = np.zeros((X.shape[0], n_class))
cat_pred_test = np.zeros((X_test.shape[0], n_class))
cat_cols = ['income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type', 'ID']
for fold in range(n_fold):
  print(f'\n----------------- Fold {fold} -----------------\n')
  train_idx, valid_idx = folds[fold]
  X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
  train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)
  valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

  model_cat = CatBoostClassifier()
  model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=100, verbose=100)
  
  cat_pred[valid_idx] = model_cat.predict_proba(X_valid)
  cat_pred_test += model_cat.predict_proba(X_test) / n_fold
  print(f'CV Log Loss Score: {log_loss(y_valid, cat_pred[valid_idx]):.6f}')
    
print(f'\tLog Loss: {log_loss(y, cat_pred):.6f}')


----------------- Fold 0 -----------------



ValueError: 'ID' is not in list

In [24]:
%%time

param_distributions = {
    "preprocessor__column__num__imputer__strategy": CategoricalDistribution(["mean","median","most_frequent"]),
    #"preprocessor__column__num__outlier__kw_args": CategoricalDistribution([{'q':0.01},{'q':0.02},{'q':0.03},{'q':0.04},{'q':0.05}]),
    #"preprocessor__column__cat__encoder__smoothing": IntDistribution(1,100),
    #"preprocessor__column__cat__encoder__min_samples_leaf": IntDistribution(1,100),
    "preprocessor__column__cat__poly__degree": IntDistribution(1,5),
    "preprocessor__selector__percentile": IntDistribution(50,100), 
    #|"classifier__alpha": IntDistribution(1,50)
}

def objective(trial: Trial) -> float:
    
    X_train, X_valid, y_train, y_valid = train_test_split(train, target, test_size=0.2)

    model = CatBoostClassifier
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        early_stopping_rounds=100,
        verbose=False,
    )

    lgb_pred = model.predict_proba(X_valid)
    log_score = log_loss(y_valid, lgb_pred)
    
    return log_score

sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="lgbm_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective, n_trials=10)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2022-11-21 14:20:32,330] A new study created in memory with name: lgbm_parameter_opt
[W 2022-11-21 14:20:32,336] Trial 0 failed because of the following error: AttributeError("'DataFrame' object has no attribute '_init_params'")
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<timed exec>", line 16, in objective
  File "C:\Users\user\anaconda3\lib\site-packages\catboost\core.py", line 5123, in fit
    params = self._init_params.copy()
  File "C:\Users\user\anaconda3\lib\site-packages\pandas\core\generic.py", line 5575, in __getattr__
    return object.__getattribute__(self, name)
AttributeError: 'DataFrame' object has no attribute '_init_params'


AttributeError: 'DataFrame' object has no attribute '_init_params'